<a href="https://colab.research.google.com/github/azavala00/Bitcoin-Price-Prediction-Using-Twitter-Sentiments-And-Currency-Fundamentals-LSTM-/blob/main/Data_Extraction_Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import numpy as np
from textblob import TextBlob
from datetime import date, datetime
from google.colab import files

Data extraction

In [ ]:
query = "(Cardano OR ADA) (#Cardano OR #ADA) until:2023-04-01 since:2021-01-01 -filter:links -filter:replies"
tweets = []
limit = 10000000000


for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.username, tweet.content, tweet.retweetCount, tweet.likeCount])
        
df1 = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet','Retweet_Count','Likes'])

<ipython-input-6-9036ac80e237>:13: DeprecatedFeatureWarning: username is deprecated, use user.username instead
  tweets.append([tweet.date, tweet.username, tweet.content, tweet.retweetCount, tweet.likeCount])
<ipython-input-6-9036ac80e237>:13: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  tweets.append([tweet.date, tweet.username, tweet.content, tweet.retweetCount, tweet.likeCount])
ERROR:snscrape.base:Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&include_ext_has_nft_avatar=1&include_ext_is_blue_verified=1&include_ext_verified_type=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_ext_limited_action_results=false&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_ext_collab_control=true&include_ext_v

ScraperException: ignored

In [ ]:
df1.to_csv('twitter_data.csv', encoding = 'utf-8-sig') 
files.download('twitter_data.csv')

In [ ]:
df1['Date'] = pd.to_datetime(df1['Date']).dt.date

In [ ]:
df1['sentiment'] = df1['Tweet'].apply(lambda review_text: TextBlob(review_text).sentiment)

In [ ]:
df1.head()

In [ ]:
df2=df1.copy()

In [ ]:
df2['polarity']= df2['sentiment'].str[0].astype(np.float).round(2)

In [ ]:
df2['subjectivity']= df2['sentiment'].str[1].astype(np.float).round(2)

In [ ]:
df2['Retweet_Count']=df2['Retweet_Count']+1
df2['Likes']=df2['Likes']+1

In [ ]:
df2.head()

In [ ]:
df_aux_sum=df2.groupby('Date').sum()
df_aux_sum.reset_index(inplace=True)
df_aux_sum.head()

In [ ]:
df3=df2.copy()

In [ ]:
del df3['User']
del df3['Tweet']

In [ ]:
df_aux = pd.merge(df3, df_aux_sum, on=["Date", "Date"])

In [ ]:
df_aux.head()

In [ ]:
df_aux['sentiment']=df_aux['Likes_x']*df_aux['polarity_x']/df_aux['Likes_y']
df_aux.head()

In [ ]:
df_aux2=df_aux.groupby('Date').sum()

In [ ]:
df=df_aux2.copy()
del df['Retweet_Count_x']
del df['Likes_x']
del df['polarity_x']
del df['subjectivity_x']
del df['Retweet_Count_y']
del df['Likes_y']
del df['polarity_y']
del df['subjectivity_y']
df.head(10)